## Project Template

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=058b58a0fc73761760cb44d6eaf391b9fa4ef5cd45f853f4a5e7f9c4064f54ab
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [1]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

from pyspark.context import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.jars.packages", 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0') \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cf458a8f-0f9e-4b22-80a0-49504de7bd31;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 171ms :: artifacts dl 3m

Be sure to start the stream on Kafka!

In [2]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType, TimestampType, DateType

schema = StructType(
      [
        StructField("name", StringType(), False),
        StructField("price", DoubleType(), False),
        StructField("timestamp", TimestampType(), False),
      ]
    )

In [3]:
kafka_server = "kafka1:9092"
from pyspark.sql.functions import from_json

lines = (spark.readStream                        # Get the DataStreamReader
  .format("kafka")                                 # Specify the source format as "kafka"
  .option("kafka.bootstrap.servers", kafka_server) # Configure the Kafka server name and port
  .option("subscribe", "stock")                       # Subscribe to the "en" Kafka topic
  .option("startingOffsets", "earliest")           # The start point when a query is started
  .option("maxOffsetsPerTrigger", 100)             # Rate limit on max offsets per trigger interval
  .load()
  .select(from_json(col("value").cast("string"), schema).alias("parsed_value"))
# Load the DataFrame
)
df = lines.select("parsed_value.*")


## The assignment starts here

You can create a

## Select the N most valuable stocks in a window

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import window, sum, rank
from pyspark.sql.window import Window
from pyspark.sql.functions import desc


In [5]:
#Create a SparkSession
Spark=SparkSession.builder.appName("StockDashboard").getOrCreate()

In [6]:
#Define an appropriate window duration and the number of most valuable stocks to select
windowDuration="5 minutes"
N = 5

In [7]:
#Grouping the data
windowed_data=df.groupBy(window("timestamp", windowDuration), "name").agg(sum("price").alias("total_value"))

In [8]:
#rank the data
Window_spec=Window.partitionBy("window").orderBy(col("total_value").desc())

In [9]:
#Creation of new column named rank
Ranked_data=windowed_data.withColumn("rank", rank().over(Window_spec))

In [10]:
#Filter the Rank column
Most_Valuable_Stocks=Ranked_data.filter(col("rank")<= N)

In [11]:
#Create an SQL view
windowed_data.createOrReplaceTempView("windowed_stocks")

In [12]:
#SQl format
sql_query = f"""
    SELECT *
    FROM windowed_stocks
    ORDER BY total_value DESC
    LIMIT {N}
"""

In [13]:
#executing the SQL query using Spark SQL
most_valuable_stocks = spark.sql(sql_query)

In [14]:
query = most_valuable_stocks.writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

23/12/18 09:46:48 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-abeb58b8-07f7-449b-8b3b-6b775dd9f6f8. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [17]:
query.awaitTermination(timeout=60)


False

In [15]:
result = spark.sql("SELECT * FROM query")

AnalysisException: Table or view not found: query; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [query]


In [16]:
result.show()

NameError: name 'result' is not defined

## Select the stocks that lost value between two windows

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, rank, lag, col
from pyspark.sql.window import Window
from pyspark.sql.functions import window, sum, rank


In [9]:
#Create a SparkSession
spark = SparkSession.builder.appName("StockMarketAnalysis").getOrCreate()

In [10]:
# Define the window duration and N value
Window_Duration = "5 minutes"
N = 5

In [11]:
# Define a watermark to handle late data
windowed_data = df.withWatermark("timestamp", "5 minutes").groupBy(window("timestamp", Window_Duration), "name").agg(sum("price").alias("total_value"))

In [12]:
# Define a window specification for ranking stocks by total value
windowSpec = Window.partitionBy("window").orderBy(col("total_value").desc())

In [13]:
# Rank the stocks by total value within each time window
ranked_data = windowed_data.withColumn("rank", rank().over(windowSpec))

In [14]:
# Filter and select the top N most valuable stocks in each window
most_valuable_stocks = ranked_data.filter(col("rank") <= N)

In [15]:
# Create a temporary view for querying
most_valuable_stocks.createOrReplaceTempView("most_valuable_stocks")

In [16]:
# Define a SQL query to select stocks that lost value between windows
sql_query = """
    SELECT a.window AS window, a.name AS name, (a.total_value - b.total_value) AS value_change
    FROM most_valuable_stocks a
    LEFT JOIN most_valuable_stocks b
    ON a.window = b.window AND a.name = b.name
    WHERE a.rank = 1 AND b.rank = 2
    AND (a.total_value - b.total_value) < 0
    ORDER BY a.window, a.name
"""


In [17]:
# Create a streaming DataFrame from the query
losing_stocks_stream = spark.sql(sql_query)

In [18]:
# Write the streaming results to a file (you can use any supported output format)
query = losing_stocks_stream.writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("losing_stocks_stream_query") \
    .start()

23/12/18 09:55:05 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-a86aa21f-5710-425a-80d8-7cfa63c8f632. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


AnalysisException: Multiple streaming aggregations are not supported with streaming DataFrames/Datasets;;
Project [window#52-T300000ms, name#53, value_change#54]
+- Sort [window#29-T300000ms ASC NULLS FIRST, name#23 ASC NULLS FIRST], true
   +- Project [window#29-T300000ms AS window#52-T300000ms, name#23 AS name#53, (total_value#34 - total_value#60) AS value_change#54, window#29-T300000ms, name#23]
      +- Filter (((rank#41 = 1) AND (rank#55 = 2)) AND ((total_value#34 - total_value#60) < cast(0 as double)))
         +- Join LeftOuter, ((window#29-T300000ms = window#59-T300000ms) AND (name#23 = name#67))
            :- SubqueryAlias a
            :  +- SubqueryAlias most_valuable_stocks
            :     +- Filter (rank#41 <= 5)
            :        +- Project [window#29-T300000ms, name#23, total_value#34, rank#41]
            :           +- Project [window#29-T300000ms, name#23, total_value#34, rank#41, rank#41]
            :              +- Window [rank(total_value#34) windowspecdefi

## Select the stock that gained the most (between windows)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import window, sum, rank, col
from pyspark.sql.window import Window


In [ ]:
#Create a SparkSession
spark = SparkSession.builder.appName("StockAnalysis").getOrCreate()

In [ ]:
# Define the window duration and the number of stocks to select
Window_Duration = "5 minutes"
N = 5

In [ ]:
# Calculate the total value of stocks in each window
windowed_data = df.groupBy(window("timestamp", Window_Duration), "name").agg(sum("price").alias("total_value"))

In [ ]:
# Define a window specification for ranking stocks by total value
Window_spec = Window.partitionBy("window").orderBy(col("total_value").desc())

In [ ]:
#Rank the stocks based on total value within each window
Ranked_data = windowed_data.withColumn("rank", rank().over(Window_spec))

In [ ]:
# Select the most valuable stocks
most_valuable_stocks = Ranked_data.filter(col("rank") <= N)

In [ ]:
# Create a temporary view for querying
windowed_data.createOrReplaceTempView("windowed_stocks")


In [46]:
# Define a SQL query to find the stocks that gained the most value between windows
sql_query = f"""
    SELECT a.window AS window, a.name AS name, (a.total_value - b.total_value) AS value_change
    FROM most_valuable_stocks a
    LEFT JOIN most_valuable_stocks b
    ON a.window = b.window AND a.name = b.name
    WHERE a.rank = 1 AND b.rank = 2
    AND (a.total_value - b.total_value) > 0
    ORDER BY a.window, a.name
"""


In [ ]:
# Create a streaming DataFrame from the query
gaining_stocks_stream = spark.sql(sql_query)

In [ ]:
# Write the streaming results to a file (you can use any supported output format)
query = gaining_stocks_stream.writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("gaining_stocks_stream_query") \
    .start()

AnalysisException: ignored

Implement a control that checks if a stock does not lose too much value in a period of time

In [47]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import window, sum, col

# Initialize a Spark session
spark = SparkSession.builder.appName("StockValueLossMonitor").getOrCreate()

In [48]:
# Define window duration and threshold for value loss
window_duration = "30 minutes"
threshold_value_loss = 0.05  # 5% loss

In [49]:
# Window data and calculate total value
windowed_data = df.groupBy(window("timestamp", window_duration), "name").agg(sum("price").alias("total_value"))

In [51]:
# Define the window specification for ranking stocks by total value
windowSpec = Window.partitionBy("window").orderBy(col("total_value").desc())

# Calculate value changes within each window
value_changes = windowed_data.withColumn("lag_total_value", lag("total_value").over(windowSpec))
value_changes = value_changes.withColumn("value_change", (col("total_value") - col("lag_total_value")) / col("total_value"))


In [52]:
# Filter stocks with significant value loss
significant_value_loss = value_changes.filter(col("value_change") < -threshold_value_loss)

In [55]:
# Define a watermark to handle late data
windowed_data = windowed_data.withWatermark("timestamp", "5 minutes")

# Filter significant value losses (for example, losses greater than -0.05 or 5%)
significant_value_loss = value_changes.filter(col("value_change") < -0.05)

# Start the streaming query
query = significant_value_loss.writeStream.outputMode("append").format("memory").queryName("significant_value_loss_query").start()
query.awaitTermination()

AnalysisException: ignored

Compute your assets

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Create a Spark session in Google Colab
spark = SparkSession.builder.appName("AssetFluctuationAnalysis").getOrCreate()

In [5]:
# Assume you have a DataFrame 'portfolio' with schema <name, amount of stocks owned>
# Replace this with your actual portfolio data
portfolio = spark.createDataFrame([("AAPL", 10), ("GOOGL", 5), ("AMZN", 7)], ["name", "amount"])



In [6]:
# Deserialize data from Kafka (assuming it's in JSON format)

# Join portfolio data with market data by stock name
joinedData = df.join(portfolio, "name", "left")

# Calculate asset value based on the current stock price and amount owned
assetValue = joinedData.withColumn("asset_value", col("price") * col("amount"))

# Group data by timestamp and calculate the total asset value
assetFluctuation = assetValue.groupBy("timestamp").agg({"asset_value": "sum"})

In [7]:
# Define an output sink (e.g., console, memory, or a file)
query = assetFluctuation.writeStream.outputMode("complete").format("console").start()

23/12/18 10:03:06 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ab2d3122-1906-40a7-b0b3-57c61f6ae010. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+-------------------+----------------+
|          timestamp|sum(asset_value)|
+-------------------+----------------+
|2023-12-15 15:30:50|            null|
|2023-12-15 15:30:44|            null|
|2023-12-15 15:30:28|            null|
|2023-12-15 15:30:20|            null|
|2023-12-15 15:30:30|            null|
|2023-12-15 15:30:06|            null|
|2023-12-15 15:30:18|            null|
|2023-12-15 15:30:16|            null|
|2023-12-15 15:30:37|            null|
|2023-12-15 15:30:51|            null|
|2023-12-15 15:30:21|            null|
|2023-12-15 15:30:46|            null|
|2023-12-15 15:30:45|            null|
|2023-12-15 15:30:17|            null|
|2023-12-15 15:30:10|            null|
|2023-12-15 15:30:52|            null|
|2023-12-15 15:30:11|            null|
|2023-12-15 15:30:26|            null|
|2023-12-15 15:30:39|            null|
|2023-12-15 15:30:09|            null|
+-----

-------------------------------------------
Batch: 1
-------------------------------------------
+-------------------+----------------+
|          timestamp|sum(asset_value)|
+-------------------+----------------+
|2023-12-15 15:30:50|            null|
|2023-12-15 15:30:44|            null|
|2023-12-15 15:30:28|            null|
|2023-12-15 15:30:20|            null|
|2023-12-15 15:30:30|            null|
|2023-12-15 15:30:06|            null|
|2023-12-15 15:30:16|            null|
|2023-12-15 15:30:18|            null|
|2023-12-15 15:30:37|            null|
|2023-12-15 15:31:15|            null|
|2023-12-15 15:31:34|            null|
|2023-12-15 15:30:51|            null|
|2023-12-15 15:31:11|            null|
|2023-12-15 15:30:21|            null|
|2023-12-15 15:30:46|            null|
|2023-12-15 15:30:45|            null|
|2023-12-15 15:30:17|            null|
|2023-12-15 15:31:29|            null|
|2023-12-15 15:31:23|            null|
|2023-12-15 15:31:19|            null|
+-----

-------------------------------------------
Batch: 2
-------------------------------------------
+-------------------+----------------+
|          timestamp|sum(asset_value)|
+-------------------+----------------+
|2023-12-15 15:32:30|            null|
|2023-12-15 15:30:50|            null|
|2023-12-15 15:30:44|            null|
|2023-12-15 15:30:28|            null|
|2023-12-15 15:30:20|            null|
|2023-12-15 15:30:30|            null|
|2023-12-15 15:30:06|            null|
|2023-12-15 15:30:16|            null|
|2023-12-15 15:30:18|            null|
|2023-12-15 15:32:20|            null|
|2023-12-15 15:32:32|            null|
|2023-12-15 15:32:22|            null|
|2023-12-15 15:30:37|            null|
|2023-12-15 15:31:15|            null|
|2023-12-15 15:31:34|            null|
|2023-12-15 15:30:51|            null|
|2023-12-15 15:31:11|            null|
|2023-12-15 15:30:21|            null|
|2023-12-15 15:30:46|            null|
|2023-12-15 15:31:50|            null|
+-----

-------------------------------------------
Batch: 3
-------------------------------------------
+-------------------+----------------+
|          timestamp|sum(asset_value)|
+-------------------+----------------+
|2023-12-15 15:32:30|            null|
|2023-12-15 15:30:50|            null|
|2023-12-15 15:33:06|            null|
|2023-12-15 15:30:44|            null|
|2023-12-15 15:30:28|            null|
|2023-12-15 15:30:20|            null|
|2023-12-15 15:32:58|            null|
|2023-12-15 15:30:30|            null|
|2023-12-15 15:30:06|            null|
|2023-12-15 15:33:19|            null|
|2023-12-15 15:30:18|            null|
|2023-12-15 15:30:16|            null|
|2023-12-15 15:32:20|            null|
|2023-12-15 15:32:32|            null|
|2023-12-15 15:32:22|            null|
|2023-12-15 15:30:37|            null|
|2023-12-15 15:31:15|            null|
|2023-12-15 15:31:34|            null|
|2023-12-15 15:30:51|            null|
|2023-12-15 15:31:11|            null|
+-----

...

-------------------------------------------
Batch: 7
-------------------------------------------
+-------------------+----------------+
|          timestamp|sum(asset_value)|
+-------------------+----------------+
|2023-12-15 15:33:06|            null|
|2023-12-15 15:35:55|            null|
|2023-12-15 15:32:30|            null|
|2023-12-15 15:30:50|            null|
|2023-12-15 15:36:24|            null|
|2023-12-15 15:30:44|            null|
|2023-12-15 15:30:28|            null|
|2023-12-15 15:34:21|            null|
|2023-12-15 15:33:28|            null|
|2023-12-15 15:30:20|            null|
|2023-12-15 15:36:07|            null|
|2023-12-15 15:32:58|            null|
|2023-12-15 15:34:40|            null|
|2023-12-15 15:30:30|            null|
|2023-12-15 15:30:06|            null|
|2023-12-15 15:36:29|            null|
|2023-12-15 15:36:06|            null|
|2023-12-15 15:33:19|            null|
|2023-12-15 15:30:18|            null|
|2023-12-15 15:30:16|            null|
+-----

-------------------------------------------
Batch: 8
-------------------------------------------
+-------------------+----------------+
|          timestamp|sum(asset_value)|
+-------------------+----------------+
|2023-12-15 15:33:06|            null|
|2023-12-15 15:35:55|            null|
|2023-12-15 15:32:30|            null|
|2023-12-15 15:30:50|            null|
|2023-12-15 15:36:24|            null|
|2023-12-15 15:30:44|            null|
|2023-12-15 15:30:28|            null|
|2023-12-15 15:34:21|            null|
|2023-12-15 15:33:28|            null|
|2023-12-15 15:30:20|            null|
|2023-12-15 15:36:07|            null|
|2023-12-15 15:32:58|            null|
|2023-12-15 15:34:40|            null|
|2023-12-15 15:30:30|            null|
|2023-12-15 15:30:06|            null|
|2023-12-15 15:36:29|            null|
|2023-12-15 15:36:06|            null|
|2023-12-15 15:33:19|            null|
|2023-12-15 15:30:18|            null|
|2023-12-15 15:30:16|            null|
+-----

-------------------------------------------
Batch: 9
-------------------------------------------
+-------------------+----------------+
|          timestamp|sum(asset_value)|
+-------------------+----------------+
|2023-12-15 15:33:06|            null|
|2023-12-15 15:35:55|            null|
|2023-12-15 15:32:30|            null|
|2023-12-15 15:30:50|            null|
|2023-12-15 15:36:24|            null|
|2023-12-15 15:30:44|            null|
|2023-12-15 15:30:28|            null|
|2023-12-15 15:34:21|            null|
|2023-12-15 15:33:28|            null|
|2023-12-15 15:30:20|            null|
|2023-12-15 15:36:07|            null|
|2023-12-15 15:32:58|            null|
|2023-12-15 15:38:24|            null|
|2023-12-15 15:34:40|            null|
|2023-12-15 15:30:30|            null|
|2023-12-15 15:30:06|            null|
|2023-12-15 15:36:29|            null|
|2023-12-15 15:38:20|            null|
|2023-12-15 15:36:06|            null|
|2023-12-15 15:33:19|            null|
+-----

-------------------------------------------
Batch: 10
-------------------------------------------
+-------------------+----------------+
|          timestamp|sum(asset_value)|
+-------------------+----------------+
|2023-12-15 15:33:06|            null|
|2023-12-15 15:35:55|            null|
|2023-12-15 15:32:30|            null|
|2023-12-15 15:30:50|            null|
|2023-12-15 15:39:07|            null|
|2023-12-15 15:36:24|            null|
|2023-12-15 15:30:44|            null|
|2023-12-15 15:30:28|            null|
|2023-12-15 15:34:21|            null|
|2023-12-15 15:33:28|            null|
|2023-12-15 15:30:20|            null|
|2023-12-15 15:36:07|            null|
|2023-12-15 15:32:58|            null|
|2023-12-15 15:38:24|            null|
|2023-12-15 15:34:40|            null|
|2023-12-15 15:30:30|            null|
|2023-12-15 15:30:06|            null|
|2023-12-15 15:38:20|            null|
|2023-12-15 15:38:48|            null|
|2023-12-15 15:36:29|            null|
+----

-------------------------------------------
Batch: 11
-------------------------------------------
+-------------------+----------------+
|          timestamp|sum(asset_value)|
+-------------------+----------------+
|2023-12-15 15:33:06|            null|
|2023-12-15 15:35:55|            null|
|2023-12-15 15:32:30|            null|
|2023-12-15 15:30:50|            null|
|2023-12-15 15:39:07|            null|
|2023-12-15 15:36:24|            null|
|2023-12-15 15:30:44|            null|
|2023-12-15 15:30:28|            null|
|2023-12-15 15:34:21|            null|
|2023-12-15 15:33:28|            null|
|2023-12-15 15:30:20|            null|
|2023-12-15 15:36:07|            null|
|2023-12-15 15:32:58|            null|
|2023-12-15 15:38:24|            null|
|2023-12-15 15:34:40|            null|
|2023-12-15 15:30:30|            null|
|2023-12-15 15:30:06|            null|
|2023-12-15 15:38:20|            null|
|2023-12-15 15:38:48|            null|
|2023-12-15 15:36:29|            null|
+----